<a href="https://colab.research.google.com/github/lpdata/fraude_bilhetagem/blob/main/notebooks/02_tratamento_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 7. Análise de comportamento de uso

Objetivo desta Etapa

Busca construir atributos derivados capazes de capturar padrões comportamentais e operacionais associados à fraude, a partir das variáveis exploradas nas etapas anteriores. Diferentemente da análise exploratória, esta fase foca na transformação e agregação de informações, visando aumentar o poder discriminante dos dados para a modelagem preditiva. As decisões tomadas aqui serão guiadas pelas hipóteses levantadas na Etapa 6 e alinhadas ao contexto do problema de negócio.

### 7.1 Preparação para engenharia de features

In [ ]:
# Ordenação temporal por cartão
df = df.sort_values(by=['id_cartao', 'ts_transacao']).reset_index(drop=True)


<small>***Comentários Letícia:** Nesta etapa inicial de preparação, os dados foram ordenados temporalmente por cartão, garantindo que todas as transformações subsequentes respeitem a sequência real de uso. Essa ordenação é fundamental para a criação de atributos comportamentais consistentes e para evitar vazamento de informação ao longo do processo de modelagem.*</small>

### 7.2 Features temporais baseadas em histórico

In [ ]:
# Calcula o tempo desde a última transação por cartão (em minutos)
# Observação: a primeira transação de cada cartão resultará em NaN
df['tempo_desde_ultima_transacao_min'] = (
    df.groupby('id_cartao')['ts_transacao']
      .diff()
      .dt.total_seconds()
      .div(60)
)

# Indicador de uso em intervalo muito curto (menos de 5 minutos)
df['uso_intervalo_curto'] = (
    df['tempo_desde_ultima_transacao_min'] < 5
).astype(int)

# Tempo desde a última transação (em horas)
df['tempo_desde_ultima_transacao_horas'] = (
    df['tempo_desde_ultima_transacao_min'] / 60
)

# Mensagem de saída
features_criadas = [
    'tempo_desde_ultima_transacao_min',
    'uso_intervalo_curto',
    'tempo_desde_ultima_transacao_horas'
]

print(f"Features temporais criadas com sucesso: {', '.join(features_criadas)}")


Features temporais criadas com sucesso: tempo_desde_ultima_transacao_min, uso_intervalo_curto, tempo_desde_ultima_transacao_horas


<small>***Comentários Letícia:** Nesta etapa, foram criadas variáveis temporais baseadas no intervalo entre transações consecutivas de cada cartão, com o objetivo de capturar padrões de uso atípicos ao longo do tempo. Essas features permitem identificar situações de reutilização em curto intervalo, frequentemente associadas a comportamentos suspeitos em sistemas de bilhetagem, e servirão de base para a análise de frequência e intensidade de uso nas etapas seguintes.*</small>

### 7.3 Features de frequência e intensidade de uso

In [ ]:
import pandas as pd

# Garantia de ordenação temporal
df = df.sort_values(by=['id_cartao', 'ts_transacao']).reset_index(drop=True)

# Quantidade de transações nas últimas 24 horas (por cartão)
def _rolling_count_24h(grp: pd.DataFrame) -> pd.Series:
    grp = grp.sort_values('ts_transacao')
    counts = (
        grp.set_index('ts_transacao')['id_transacao']
           .rolling('24h')
           .count()
           .to_numpy()
    )
    return pd.Series(counts, index=grp.index)

df['qtd_transacoes_24h'] = (
    df.groupby('id_cartao', group_keys=False)
      .apply(_rolling_count_24h)
      .astype(int)
)

# Indicador de uso intenso nas últimas 24 horas
df['uso_intenso_24h'] = (df['qtd_transacoes_24h'] > 4).astype(int)

# Quantidade de transações no mesmo dia (por cartão)
df['data_transacao'] = df['ts_transacao'].dt.floor('D')

df['qtd_transacoes_dia'] = (
    df.groupby(['id_cartao', 'data_transacao'])['id_transacao']
      .transform('count')
)

features_criadas = [
    'qtd_transacoes_24h',
    'uso_intenso_24h',
    'data_transacao',
    'qtd_transacoes_dia'
]

print(f"Features de frequência criadas com sucesso: {', '.join(features_criadas)}")



Features de frequência criadas com sucesso: qtd_transacoes_24h, uso_intenso_24h, data_transacao, qtd_transacoes_dia


/tmp/ipython-input-1427267726.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rolling_count_24h)


<small>***Comentários Letícia:** Nesta etapa, foram criadas variáveis de frequência e intensidade de uso baseadas na concentração de transações em janelas temporais curtas e no ciclo diário de utilização do cartão. Essas features permitem identificar padrões de uso excessivo ou atípico, complementando as variáveis temporais da etapa anterior e fornecendo sinais relevantes sobre comportamentos que podem estar associados a fraudes no contexto de bilhetagem.*</small>

### 7.4 Features de consistência comportamental

In [ ]:
# Garantia de ordenação temporal
df = df.sort_values(by=['id_cartao', 'ts_transacao']).reset_index(drop=True)

# Indicador de repetição da mesma linha de ônibus (transação consecutiva do mesmo cartão)
df['linha_onibus_repetida'] = (
    df['linha_onibus'].eq(df.groupby('id_cartao')['linha_onibus'].shift(1))
).astype(int)

# Indicador de repetição do mesmo dispositivo (validador) na transação consecutiva do mesmo cartão
df['dispositivo_repetido'] = (
    df['id_dispositivo'].eq(df.groupby('id_cartao')['id_dispositivo'].shift(1))
).astype(int)

# Quantidade de linhas distintas utilizadas pelo cartão no mesmo dia
# (mantive data_transacao como datetime "floor" para consistência)
df['data_transacao'] = df['ts_transacao'].dt.floor('D')

df['qtd_linhas_distintas_dia'] = (
    df.groupby(['id_cartao', 'data_transacao'])['linha_onibus']
      .transform('nunique')
)

features_criadas = [
    'linha_onibus_repetida',
    'dispositivo_repetido',
    'data_transacao',
    'qtd_linhas_distintas_dia'
]

print(f"Features de consistência comportamental criadas com sucesso: {', '.join(features_criadas)}")


Features de consistência comportamental criadas com sucesso: linha_onibus_repetida, dispositivo_repetido, data_transacao, qtd_linhas_distintas_dia


<small>***Comentários Letícia:** Nesta etapa, foram criadas variáveis de consistência comportamental com foco na repetição de contexto operacional ao longo do uso do cartão. As features desenvolvidas permitem identificar padrões de reutilização consecutiva da mesma linha ou do mesmo dispositivo, bem como a diversidade de linhas utilizadas em um único dia, contribuindo para a caracterização de comportamentos recorrentes ou atípicos que podem estar associados à ocorrência de fraude.*</small>

### 7.5 Features operacionais e de contexto

In [ ]:
import numpy as np
import pandas as pd

# Indicador de idade inválida/suspeita
df['idade_suspeita'] = (
    (df['idade_usuario'] < 0) | (df['idade_usuario'] > 110) | (df['idade_usuario'].isna())
).astype(int)

# Padronização de feriado para binário (0/1)
feriado_map = {
    'sim': 1, 's': 1, 'true': 1, '1': 1,
    'não': 0, 'nao': 0, 'n': 0, 'false': 0, '0': 0
}

feriado_str = df['feriado'].astype(str).str.strip().str.lower()
df['feriado_bin'] = feriado_str.map(feriado_map)

# Flag auxiliar para diagnosticar valores não mapeados
df['feriado_nao_mapeado'] = df['feriado_bin'].isna().astype(int)

# Preenchimento conservador (não mapeado -> 0)
df['feriado_bin'] = df['feriado_bin'].fillna(0).astype(int)

# Faixas de temperatura externa
df['temp_faixa'] = pd.cut(
    df['temp_externa'],
    bins=[-np.inf, 15, 20, 25, 30, np.inf],
    labels=['<=15', '15-20', '20-25', '25-30', '>30']
)

# Faixas do valor da transação
# Usa labels=False para evitar conflito quando duplicates='drop' reduzir o número de bins
bins_valor = pd.qcut(df['valor_transacao'], q=4, duplicates='drop', labels=False)

# Converte para rótulos "Q1..Qk" dinamicamente, conforme o número real de bins
if bins_valor.isna().all():
    # Caso extremo: coluna toda NaN (improvável), mantem como NaN
    df['valor_transacao_faixa'] = np.nan
else:
    n_bins = int(bins_valor.max() + 1)
    df['valor_transacao_faixa'] = pd.Categorical(
        bins_valor.map(lambda x: f"Q{int(x)+1}" if pd.notna(x) else np.nan),
        categories=[f"Q{i}" for i in range(1, n_bins + 1)],
        ordered=True
    )

features_criadas = [
    'idade_suspeita',
    'feriado_bin',
    'feriado_nao_mapeado',
    'temp_faixa',
    'valor_transacao_faixa'
]

print(f"Features operacionais e de contexto criadas com sucesso: {', '.join(features_criadas)}")


Features operacionais e de contexto criadas com sucesso: idade_suspeita, feriado_bin, feriado_nao_mapeado, temp_faixa, valor_transacao_faixa


<small>***Comentários Letícia:** Nesta etapa, foram criadas variáveis operacionais e de contexto com foco na padronização e na redução de ruído dos dados, incluindo indicadores de qualidade cadastral, normalização de informações de feriado e discretização de variáveis contínuas em faixas interpretáveis. Essas transformações visam tornar os dados mais estáveis e consistentes para a modelagem, preservando a interpretabilidade e facilitando a captura de padrões relevantes nas etapas seguintes.*</small>

### 7.6 Consolidação das features criadas

In [ ]:
# Lista completa de features criadas na Etapa 7
features_etapa_7 = [
    # 7.2 - Features temporais
    'tempo_desde_ultima_transacao_min',
    'tempo_desde_ultima_transacao_horas',
    'uso_intervalo_curto',

    # 7.3 - Frequência e intensidade
    'qtd_transacoes_24h',
    'uso_intenso_24h',
    'qtd_transacoes_dia',

    # 7.4 - Consistência comportamental
    'linha_onibus_repetida',
    'dispositivo_repetido',
    'qtd_linhas_distintas_dia',

    # 7.5 - Operacionais e contexto
    'idade_suspeita',
    'feriado_bin',
    'feriado_nao_mapeado',
    'temp_faixa',
    'valor_transacao_faixa'
]

# Verificação de existência das features
features_ausentes = [f for f in features_etapa_7 if f not in df.columns]

if features_ausentes:
    print("⚠️ Atenção: as seguintes features não foram encontradas no DataFrame:")
    for f in features_ausentes:
        print(f"- {f}")
else:
    print("✅ Todas as features da Etapa 7 foram criadas e consolidadas com sucesso.")

# Cria DataFrame consolidado para as próximas etapas
df_features = df.copy()

print(f"Total de features consolidadas na Etapa 7: {len(features_etapa_7)}")

# Visualização rápida das features criadas
print("\nAmostra das features criadas (primeiras 5 linhas):")
display(df_features[features_etapa_7].head())


✅ Todas as features da Etapa 7 foram criadas e consolidadas com sucesso.
Total de features consolidadas na Etapa 7: 14

Amostra das features criadas (primeiras 5 linhas):


,tempo_desde_ultima_transacao_min,tempo_desde_ultima_transacao_horas,uso_intervalo_curto,qtd_transacoes_24h,uso_intenso_24h,qtd_transacoes_dia,linha_onibus_repetida,dispositivo_repetido,qtd_linhas_distintas_dia,idade_suspeita,feriado_bin,feriado_nao_mapeado,temp_faixa,valor_transacao_faixa
0,NaN,NaN,0,1,0,1,0,0,1,0,0,0,25-30,Q1
1,7149.283333,119.154722,0,1,0,1,0,0,1,0,0,0,>30,Q1
2,NaN,NaN,0,1,0,1,0,0,1,0,0,0,25-30,Q1
3,3666.900000,61.115000,0,1,0,1,0,0,1,0,0,0,25-30,Q1
4,14909.300000,248.488333,0,1,0,1,0,0,1,0,0,0,25-30,Q1


<small>***Comentários Letícia:** Nesta etapa final, todas as features desenvolvidas ao longo da engenharia de atributos foram organizadas e consolidadas, garantindo sua disponibilidade e consistência para as etapas seguintes do projeto. Esse processo permite uma transição estruturada para o pré-processamento e a modelagem, assegurando rastreabilidade das variáveis criadas e facilitando a manutenção e a evolução do pipeline analítico.*</small>